# Kicking the Tires on BGE-M3

Experiments from the blog post. Testing all three modes against 100K property listings.

**Why Python instead of SQL Server?** SQL Server 2025's VECTOR_DISTANCE only works with dense embeddings. To test sparse and ColBERT, I pulled data into Python and used FlagEmbedding directly.

**Data source:** [SemanticSonarDB](https://github.com/MrJoeSack/sqlserver-sample-databases/tree/master/semantic-sonar-db) - 100K synthetic property listings for SQL Server.

In [ ]:
# Install dependencies (run once)
# pip install FlagEmbedding pyodbc numpy

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

import time
import pyodbc
import numpy as np
from FlagEmbedding import BGEM3FlagModel

def get_conn():
    """Connect to SemanticSonarDB. Update connection string for your environment."""
    return pyodbc.connect(
        "DRIVER={ODBC Driver 18 for SQL Server};"
        "SERVER=localhost;DATABASE=SemanticSonarDB;"
        "TrustServerCertificate=yes;Trusted_Connection=yes;"
    )

print("Loading BGE-M3...")
start = time.time()
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)
print(f"Loaded in {time.time() - start:.1f}s")

In [ ]:
# Load test data
conn = get_conn()
cursor = conn.cursor()

# 50 random listings for tests
cursor.execute("""
    SELECT TOP 50 property_id, listing_description
    FROM PropertyListings
    ORDER BY NEWID()
""")
docs = [(r[0], r[1]) for r in cursor.fetchall()]
conn.close()

print(f"Loaded {len(docs)} documents")

In [ ]:
# Scoring functions for each mode

def dense_score(q_emb, d_emb):
    """Cosine similarity between query and document dense vectors."""
    return float(q_emb @ d_emb.T)

def sparse_score(q_sparse, d_sparse):
    """Sum of minimum weights for tokens appearing in both query and document."""
    score = 0.0
    for token in q_sparse:
        if token in d_sparse:
            score += min(q_sparse[token], d_sparse[token])
    return score

def colbert_score(q_vecs, d_vecs):
    """MaxSim: for each query token, find max similarity to any document token, then sum."""
    return model.colbert_score(q_vecs, d_vecs)

def rank_all_modes(query, docs, doc_output):
    """Rank documents by each mode separately. Returns dict with sorted results per mode."""
    q_out = model.encode([query], return_dense=True, return_sparse=True, return_colbert_vecs=True)
    
    results = {'dense': [], 'sparse': [], 'colbert': []}
    for i, (pid, text) in enumerate(docs):
        results['dense'].append((pid, dense_score(q_out['dense_vecs'][0], doc_output['dense_vecs'][i])))
        results['sparse'].append((pid, sparse_score(q_out['lexical_weights'][0], doc_output['lexical_weights'][i])))
        results['colbert'].append((pid, colbert_score(q_out['colbert_vecs'][0], doc_output['colbert_vecs'][i])))
    
    for mode in results:
        results[mode] = sorted(results[mode], key=lambda x: x[1], reverse=True)
    return results

---
## Three Methods, Same Query

Each mode picks a different winner. Didn't expect that.

In [ ]:
# Encode all docs with all three modes
print("Encoding documents...")
doc_output = model.encode([d[1] for d in docs], return_dense=True, return_sparse=True, return_colbert_vecs=True)

query = "cozy home with fireplace"
results = rank_all_modes(query, docs, doc_output)

print(f'\n"{query}"')
print(f"  Dense:   property {results['dense'][0][0]} (score: {results['dense'][0][1]:.3f})")
print(f"  Sparse:  property {results['sparse'][0][0]} (score: {results['sparse'][0][1]:.3f})")
print(f"  ColBERT: property {results['colbert'][0][0]} (score: {results['colbert'][0][1]:.3f})")

---
## French Query, English Docs

This one was interesting.

In [ ]:
queries = [
    ("English", "cozy home with fireplace"),
    ("French", "maison confortable avec cheminee"),
]

for lang, query in queries:
    q_out = model.encode([query], return_dense=True, return_sparse=True, return_colbert_vecs=True)
    
    # Get max scores across all docs
    dense_max = max(dense_score(q_out['dense_vecs'][0], doc_output['dense_vecs'][i]) for i in range(len(docs)))
    sparse_max = max(sparse_score(q_out['lexical_weights'][0], doc_output['lexical_weights'][i]) for i in range(len(docs)))
    
    print(f'{lang}: "{query}"')
    print(f"  Dense:  {dense_max:.3f}")
    print(f"  Sparse: {sparse_max:.3f}")
    print()

Sparse found nothing. No French words in English listings means no lexical overlap. Dense understood "cheminee" maps to fireplace semantically.

---
## Other Observations

In [ ]:
# How much slower is ColBERT scoring?
q_out = model.encode(["test query"], return_dense=True, return_sparse=True, return_colbert_vecs=True)

start = time.time()
for _ in range(100):
    for i in range(len(docs)):
        _ = dense_score(q_out['dense_vecs'][0], doc_output['dense_vecs'][i])
dense_time = time.time() - start

start = time.time()
for _ in range(100):
    for i in range(len(docs)):
        _ = colbert_score(q_out['colbert_vecs'][0], doc_output['colbert_vecs'][i])
colbert_time = time.time() - start

print(f"Scoring {len(docs)} docs (100 iterations):")
print(f"  Dense:   {dense_time:.2f}s")
print(f"  ColBERT: {colbert_time:.2f}s")
print(f"  ColBERT is ~{colbert_time/dense_time:.0f}x slower")